# packages

In [10]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# settings

In [11]:
# default column number pandas
pd.options.display.max_columns = 200

# plot style
sns.set(palette = "colorblind", font_scale = 1.5)

# data

In [16]:
hpi = pd.read_excel("/Users/nicholasmichalak/sandbox/hpi-analysis/data/HPI_AT_BDL_cbsa.xlsx", skiprows = 6)

# better column names
## function
def adjust_colname(x: str, to_replace: dict):
    for s, r in to_replace.items():
        # replace
        x = x.replace(s, r)
    return x.lower()

# replace names
hpi.columns = [adjust_colname(col, {" ": "_", "%": "pct", "(": "", ")": ""}) for col in hpi.columns]

# see it
hpi

,cbsa,name,year,annual_change_pct,hpi,hpi_with_1990_base,hpi_with_2000_base
0,1,Alabama(non CBSA areas),1985,.,100.00,91.76,63.20
1,1,Alabama(non CBSA areas),1986,5.89,105.89,97.16,66.92
2,1,Alabama(non CBSA areas),1987,0.78,106.71,97.92,67.44
3,1,Alabama(non CBSA areas),1988,-0.49,106.19,97.44,67.11
4,1,Alabama(non CBSA areas),1989,5.04,111.55,102.35,70.49
...,...,...,...,...,...,...,...
39669,49780,"Zanesville, OH",2017,3.44,278.12,200.33,119.62
39670,49780,"Zanesville, OH",2018,6.62,296.54,213.60,127.54
39671,49780,"Zanesville, OH",2019,4.51,309.92,223.24,133.30
39672,49780,"Zanesville, OH",2020,3.45,320.61,230.93,137.89
